# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import googlemaps 
import json 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:

weather_data_file = "cities.csv"
weather_df = pd.read_csv(weather_data_file)
weather_df.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,0,[1],provideniya,9,RU,1672615938,77,64.3833,-173.3000,-0.94,12.95
1,1,[2],cherskiy,93,RU,1672615938,99,68.7500,161.3000,-45.92,5.06
2,2,[4],hobart,0,AU,1672615802,71,-42.8794,147.3294,77.22,5.75
3,3,[5],jalu,78,LY,1672615939,66,29.0331,21.5482,54.10,5.50
4,4,[6],ushuaia,40,AR,1672615939,50,-54.8000,-68.3000,55.06,12.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# Plot Heatmap
gmaps.configure(api_key = g_key)

fig = gmaps.figure()
locations = weather_df[["Latitude", "Longitude"]]
 
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"], 
                                 dissipating=False,
                                 point_radius=5)


# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't satisfy all three conditions. You want to be sure the weather is ideal.
hotel_df = weather_df[(weather_df["Max Temp"] < 80) & (weather_df["Max Temp"] > 70) & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0)] .copy()


hotel_df['Hotel Name'] = ''
hotel_df.head()

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
2,2,[4],hobart,0,AU,1672615802,71,-42.8794,147.3294,77.22,5.75,
57,57,[61],tomatlan,0,MX,1672615955,60,19.9333,-105.2500,78.31,6.58,
240,240,[264],saint-pierre,0,RE,1672616012,69,-21.3393,55.4781,74.88,4.61,
305,305,[337],san angelo,0,US,1672616033,30,31.4638,-100.4370,79.02,8.05,
333,333,[370],vallenar,0,CL,1672616041,44,-28.5708,-70.7581,73.67,4.34,


### Hotel Map
* Store into variable named `hotel_hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# gmap = googlemaps.Client(key= g_key)


# hotel_list = []
# for index, row in hotel_df.iterrows():
#     lat = row['Latitude']
#     lng = row['Longitude']
#     nearest_hotel = gmap.places_nearby(location=(lat, lng), radius=5000, type='hotel')
#     hotel_list.append(nearest_hotel['results'][0]['name'])
#     # hotel_hotel_df.loc[index, 'Hotel Name'] = nearest_hotel




In [25]:



base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
key = g_key


for index, row in hotel_df.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']
    params = {
    'location': f"{lat},{lng}",
    'radius': '5000',
    'type': 'lodging',
    'key': key
    }
    response = requests.get(base_url, params=params)
    data = response.json()

    hotel_df.at[index, 'Hotel Name'] = data['results'][0]['name']

In [26]:
hotel_df

,Unnamed: 0,City ID,City Name,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
2,2,[4],hobart,0,AU,1672615802,71,-42.8794,147.3294,77.22,5.75,St Ives Hobart Accommodation Tasmania
57,57,[61],tomatlan,0,MX,1672615955,60,19.9333,-105.2500,78.31,6.58,El Salvial
240,240,[264],saint-pierre,0,RE,1672616012,69,-21.3393,55.4781,74.88,4.61,Alize Plage
305,305,[337],san angelo,0,US,1672616033,30,31.4638,-100.4370,79.02,8.05,"Motel 6 San Angelo, TX"
333,333,[370],vallenar,0,CL,1672616041,44,-28.5708,-70.7581,73.67,4.34,Humacao Bed & Breakfast
350,350,[387],lima,0,PE,1672615863,78,-12.0432,-77.0282,70.05,8.05,Hotel Kamana
475,475,[524],axim,0,GH,1672616085,72,4.8699,-2.2405,78.24,5.82,Axim Beach Resort And Spa
568,568,[624],topolobampo,0,MX,1672616115,56,25.6000,-109.0500,73.31,8.05,Hotel Marina Topolobampo


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [34]:

# Add marker layer ontop of heat map

hotels = [(row['Latitude'], row['Longitude']) for index, row in hotel_df.iterrows()]
marker_layer = gmaps.marker_layer(hotels, label=[row['Hotel Name'] for index, row in hotel_df.iterrows()])
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))